In [1]:
%pylab inline
import numpy as np
import scipy.linalg as la
import os,sys,inspect

data_path = '../pulses/output_pulses/'

from quantum_optimal_control.helper_functions.grape_functions import *
from quantum_optimal_control.main_grape.grape import Grape

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def run_Hadamard(qubit_num):


    #Defining time scales
    total_time = qubit_num *2.0
    steps = qubit_num * 10

    # Choose optimizing State transfer or Unitary gate
    state_transfer = False
    
    # Choose whether include intermediate state evolution as part of the graph optimization
    use_inter_vecs = False
    
    #Defining H0
    qubit_state_num = 2
    freq_ge = 0#GHz
    g = 2*np.pi*0.1 #GHz

    Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
    Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
    sigma_z   = np.diag(np.arange(-1,qubit_state_num,2))
    I_q = np.identity(qubit_state_num)
    

    g_op = nn_chain_kron(sigma_z, I_q, qubit_num, qubit_state_num)

    H0 = g * g_op
    

    # Defining dressed info
    is_dressed = False
#     w_c, v_c, dressed_id = get_dressed_info(H0)
#     dressed_info = {'dressed_id':dressed_id, 'eigenvectors':v_c, 'eigenvalues':w_c,'is_dressed':is_dressed}
    
    dressed_info = None


    #Defining Concerned states (starting states)

    psi0 = concerned(qubit_num,qubit_state_num)


    #Defining states to include in the drawing of occupation
    states_draw_list = range(qubit_state_num**qubit_num)
    states_draw_names =[]
    for ii in states_draw_list:
        states_draw_names.append(Basis(ii,qubit_num,qubit_state_num))

    #Defining U (Target)

    if is_dressed:    
        U = dressed_unitary(Hadamard(qubit_num),v_c,dressed_id)
    else:   
        U= Hadamard(qubit_num)

    #print U    
    
    if state_transfer:
        target_vec_list = []

        for ii in psi0:
            target_vec = np.dot(U,v_c[:,get_state_index(ii,dressed_id)])
            target_vec_list.append(target_vec)

        U = target_vec_list


    #Defining U0 (Initial)
    q_identity = np.identity(qubit_state_num**qubit_num)
    U0= q_identity

    #Defining control Hs

    Hops = []
    Hnames = []
    ops_max_amp = []
    max_amp=2*np.pi*0.5
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'y',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    #Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=max_amp)
    
    #print Hops[0]
    print Hnames

    #Defining convergence parameters
    max_iterations = 1000
    decay = 1000 #max_iterations/2
    convergence = {'rate':0.01, 'update_step':10 ,'max_iterations':max_iterations,\
                   'conv_target':1e-3,'learning_rate_decay':decay,'min_grad':1e-60}

    reg_coeffs = {}
    
    #import h5py
    #with h5py.File('/home/nelson/Simulations/GRAPE-Data/spin_chain_hadamard/00002_gpu_spin_chain_hadamard_N9.h5','r') as hf:
    #    u0 = np.array(hf.get('uks'))[-1]
    
    u0=None
    
    # set data path, the folder must be an existing directory

    uks,U_final = Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence=convergence, draw = [states_draw_list,states_draw_names],  

                        show_plots = False, use_gpu = True,sparse_H=False,sparse_U=False,state_transfer = state_transfer, use_inter_vecs = use_inter_vecs,
           unitary_error = 1e-8,  maxA=ops_max_amp,Taylor_terms =[20,0],initial_guess=u0,
                       dressed_info = dressed_info, method = 'ADAM', reg_coeffs=reg_coeffs, file_name='dense_gpu_spin_chain_hadamard_N%d' %(qubit_num), 
                        data_path = data_path)

In [3]:
run_Hadamard(qubit_num=2)

['xi', 'ix', 'yi', 'iy']
data saved at: ../pulses/output_pulses/00000_dense_gpu_spin_chain_hadamard_N2.h5
Using 20 Taylor terms and 0 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.56e-01; Runtime: 0.0s; Iterations = 0, grads =   1.027e-01, unitary_metric = 1.00000
Error = :3.75e-01; Runtime: 0.1s; Iterations = 10, grads =   6.978e-01, unitary_metric = 1.00000
Error = :1.17e-01; Runtime: 0.1s; Iterations = 20, grads =   3.339e-01, unitary_metric = 1.00000
Error = :5.31e-02; Runtime: 0.2s; Iterations = 30, grads =   5.725e-02, unitary_metric = 1.00000
Error = :2.54e-02; Runtime: 0.2s; Iterations = 40, grads =   2.353e-02, unitary_metric = 1.00000
Error = :9.75e-03; Runtime: 0.2s; Iterations = 50, grads =   1.132e-02, unitary_metric = 1.00000
Error